In [1]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from rank_bm25 import BM25Okapi
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
# Hàm tự động lấy links từ bài viết và nhiều chỉ mục trong mục lục
def auto_getlink(base_url: str, num_pages: int = 20):
    links = []
    
    for page in range(1, num_pages):
        # Tạo URL cho từng trang
        url = f"{base_url}-p{page}"  # Cấu trúc URL mới
        
        # Gửi yêu cầu HTTP
        response = requests.get(url)

        # Kiểm tra xem yêu cầu có thành công không
        if response.status_code == 200:
            # Phân tích cú pháp HTML
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Tìm tất cả các thẻ <a> có chứa link bài viết
            for a in soup.find_all('a', class_="count_cmt", href=True):
                # Lấy link
                full_link = a['href']
                
                # Thêm link đầy đủ vào danh sách
                if not full_link.startswith('http'):
                    full_link = "https://vnexpress.net" + full_link
                
                # Thêm vào danh sách link
                links.append(full_link)
        
        else:
            print(f"Không thể truy cập trang {url}. Mã lỗi: {response.status_code}")

    # Loại bỏ các link trùng nhau
    links = list(set(links))
    
    # Trả về danh sách
    return links

In [3]:
# Hàm tự động lấy dữ liệu từ 1 bài viết
def vnexpress(url: str):
    
    # Gửi yêu cầu HTTP
    response = requests.get(url)
    
    # Kiểm tra xem yêu cầu có thành công không
    if response.status_code == 200:
        # Phân tích cú pháp HTML
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Tìm tiêu đề bài báo
        # title = soup.find('h1', class_='title-detail').get_text(strip=True)
        title_element = soup.find('h1', class_='title-detail')
        title = title_element.get_text(strip=True) if title_element else ""
    
    # Trả về Title
    return str(title)

In [4]:
# Hàm auto craw data
def auto_gettext(url: str, num_pages: int):
    links = auto_getlink(url, num_pages)
    news = []
    for link in links:
        news.append(vnexpress(link))
    return links, news

In [5]:
# Thử nghiệm
link = "https://vnexpress.net/thoi-su/chinh-tri"
links, news = auto_gettext(url=link, num_pages=2)

In [20]:
# Xử lí dữ liệu
tokenized_real_news = [doc.split(" ") for doc in news]  # Token hóa bằng cách tách các từ

In [22]:
# Khởi tạo mô hình BM25
bm25 = BM25Okapi(tokenized_real_news)

In [23]:
# Nhập nội dung tin tức giả
fake_news_content = "Từ nay, tăng thuế lên 20%"
tokenized_fake_news = fake_news_content.split(" ")

In [24]:
# Tìm kiếm các tin tức thật
scores = bm25.get_scores(tokenized_fake_news)

In [25]:
# Lấy các chỉ số của các tin tức thật có điểm số cao nhất
top_n = 5  # Số lượng tin tức thật bạn muốn hiển thị
top_indices = scores.argsort()[-top_n:][::-1]  # Lấy chỉ số của top N tin tức thật

In [26]:
# Hiển thị kết quả
print("Các tin tức thật tương tự:")
for index in top_indices:
    print(f"-[{index + 1}] Tiêu đề: {news[index]}")

Các tin tức thật tương tự:
-[4] Tiêu đề: 'Nên tăng lương hưu từ năm 2025'
-[15] Tiêu đề: Thủ tướng đề nghị tập đoàn bán lẻ hàng đầu thế giới tăng bán hàng Việt
-[29] Tiêu đề: Thủ tướng sắp thăm UAE, Qatar, Arab Saudi
-[2] Tiêu đề: Đà Nẵng có tân Chủ tịch UBND huyện Hoàng Sa
-[3] Tiêu đề: Đề xuất biện pháp ngăn tẩu tán tài sản trong các đại án


In [27]:
articles = []
for index in top_indices:
    articles.append({"title": news[index], "link": links[index]})

In [28]:
articles

[{'title': "'Nên tăng lương hưu từ năm 2025'",
  'link': 'https://vnexpress.net/nen-tang-luong-huu-tu-nam-2025-4808713.html#box_comment_vne'},
 {'title': 'Thủ tướng đề nghị tập đoàn bán lẻ hàng đầu thế giới tăng bán hàng Việt',
  'link': 'https://vnexpress.net/thu-tuong-de-nghi-tap-doan-ban-le-hang-dau-the-gioi-tang-ban-hang-viet-4809044.html#box_comment_vne'},
 {'title': 'Thủ tướng sắp thăm UAE, Qatar, Arab Saudi',
  'link': 'https://vnexpress.net/thu-tuong-sap-tham-uae-qatar-arab-saudi-4808804.html#box_comment_vne'},
 {'title': 'Đà Nẵng có tân Chủ tịch UBND huyện Hoàng Sa',
  'link': 'https://vnexpress.net/da-nang-co-tan-chu-tich-ubnd-huyen-hoang-sa-4808339.html#box_comment_vne'},
 {'title': 'Đề xuất biện pháp ngăn tẩu tán tài sản trong các đại án',
  'link': 'https://vnexpress.net/de-xuat-bien-phap-ngan-tau-tan-tai-san-trong-cac-dai-an-4810144.html#box_comment_vne'}]

In [2]:
# Thực hiện lấy links và content thông qua database
# Tạo kết nối đến MySQL
connection = mysql.connector.connect(
    host="localhost",     # Thường là localhost
    user="root",          # Người dùng MySQL, mặc định là root
    password="",          # Mật khẩu MySQL, để trống nếu không có
    database="pbl6"  # Tên cơ sở dữ liệu
)

# Tạo con trỏ để thực hiện truy vấn
cursor = connection.cursor()

# Thực hiện truy vấn
cursor.execute("SELECT Title, Source FROM tbl_news")

# Lấy tất cả kết quả từ truy vấn
rows = cursor.fetchall()

links = []
news = []

# Xử lý kết quả
for row in rows:
    content, source = row
    news.append(content)
    links.append(source)
    # print(f"Content: {content}, Source: {source}")

# Đóng kết nối
cursor.close()
connection.close()

In [3]:
# Xử lí dữ liệu
tokenized_real_news = [doc.split(" ") for doc in news]  # Token hóa bằng cách tách các từ

# Nhập nội dung tin tức giả
fake_news_content = "Từ nay, tăng thuế lên 20%"
tokenized_fake_news = fake_news_content.split(" ")

# Khởi tạo mô hình BM25
bm25 = BM25Okapi(tokenized_real_news)

# Tìm kiếm các tin tức thật
scores = bm25.get_scores(tokenized_fake_news)

# Lấy các chỉ số của các tin tức thật có điểm số cao nhất
top_n = 5  # Số lượng tin tức thật bạn muốn hiển thị
top_indices = scores.argsort()[-top_n:][::-1]  # Lấy chỉ số của top N tin tức thật

# Hiển thị kết quả
print("Các tin tức thật tương tự:")
for index in top_indices:
    print(f"-[{index + 1}] Links: {links[index]}")

Các tin tức thật tương tự:
-[10082] Links: https://vietnamnet.vn/de-xuat-giam-thue-thu-nhap-doanh-nghiep-dau-khi-tu-25-xuong-20-2043454.html
-[5973] Links: https://vietnamnet.vn/bi-thu-tp-hcm-tu-day-tang-truong-0-7-len-5-81-la-su-no-luc-cua-toan-thanh-pho-2236086.html
-[838] Links: https://thanhnien.vn/ngan-chan-gian-lan-thue-gia-tri-gia-tang-185240423230633843.htm
-[7575] Links: https://vietnamnet.vn/vinh-phuc-thanh-lap-moi-duoc-69-to-chuc-dang-trong-doanh-nghiep-ngoai-nha-nuoc-2214683.html
-[4660] Links: https://vietnamnet.vn/giam-thue-gia-tri-gia-tang-xuong-8-tu-1-7-den-31-12-2024-2296922.html


In [5]:
from database.Pbl6_database import *

my_database = Pbl6_database()
fake_news_content = "Từ nay, tăng thuế lên 20%"

articles = my_database.rag_news(fake_news_content)

In [6]:
articles

[{'title': 'Đề xuất giảm thuế thu nhập doanh nghiệp dầu khí từ 25% xuống 20%',
  'link': 'https://vietnamnet.vn/de-xuat-giam-thue-thu-nhap-doanh-nghiep-dau-khi-tu-25-xuong-20-2043454.html'},
 {'title': 'Bí thư TP.HCM: Từ đáy tăng trưởng 0,7 lên 5,81% là sự nỗ lực của toàn thành phố',
  'link': 'https://vietnamnet.vn/bi-thu-tp-hcm-tu-day-tang-truong-0-7-len-5-81-la-su-no-luc-cua-toan-thanh-pho-2236086.html'},
 {'title': 'Ngăn chặn gian lận thuế giá trị gia tăng',
  'link': 'https://thanhnien.vn/ngan-chan-gian-lan-thue-gia-tri-gia-tang-185240423230633843.htm'},
 {'title': 'Vĩnh Phúc: Từ đầu nhiệm kỳ đến nay, thành lập mới 69 tổ chức Đảng trong doanh nghiệp ngoài Nhà nước',
  'link': 'https://vietnamnet.vn/vinh-phuc-thanh-lap-moi-duoc-69-to-chuc-dang-trong-doanh-nghiep-ngoai-nha-nuoc-2214683.html'},
 {'title': 'Giảm thuế giá trị gia tăng xuống 8% từ 1/7 đến 31/12/2024',
  'link': 'https://vietnamnet.vn/giam-thue-gia-tri-gia-tang-xuong-8-tu-1-7-den-31-12-2024-2296922.html'}]